<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [4]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [5]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

In [7]:
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


Now, you can take a look at what are the coordinates for each site.


In [8]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [9]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [10]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [11]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
# Añadir un círculo y marcador para cada sitio de lanzamiento
for _, row in launch_sites_df.iterrows():
    # Extraer coordenadas y nombre del sitio
    site_name = row['Launch Site']
    coordinate = [row['Lat'], row['Long']]
    
    # Crear un círculo en el sitio de lanzamiento
    circle = folium.Circle(coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup(site_name))
    # Crear un marcador en el sitio de lanzamiento
    marker = folium.map.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % site_name,
        )
    )
    # Añadir círculo y marcador al mapa
    site_map.add_child(circle)
    site_map.add_child(marker)

# Mostrar el mapa
site_map

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


## Proximidad a la línea del ecuador:

    La línea del ecuador se encuentra a una latitud de 0 grados. Los sitios de lanzamiento más cercanos al ecuador tienden a aprovechar la velocidad de rotación de la Tierra, lo que puede ayudar a reducir el consumo de combustible necesario para alcanzar la órbita.
    Observando las coordenadas de los sitios de lanzamiento:
        CCAFS SLC 40: Aproximadamente 28.5°N, 80.5°W
        KSC LC 39A: Aproximadamente 28.6°N, 80.6°W
        VAFB SLC 4E: Aproximadamente 34.6°N, 120.6°W
        STLS: Aproximadamente 30.4°N, 97.6°W
    Ninguno de estos sitios está muy cerca del ecuador (0° de latitud). Están en latitudes más altas, lo que significa que no están optimizados exclusivamente para lanzamientos ecuatoriales. Sin embargo, están en latitudes que aún pueden beneficiarse de la rotación de la Tierra, aunque en menor medida.

## Proximidad a la costa:

    La proximidad a la costa es importante porque los lanzamientos sobre el océano son más seguros en caso de fallos, ya que los desechos de los cohetes caen en el agua en lugar de en áreas pobladas.
    Observando las coordenadas y ubicaciones de los sitios de lanzamiento:
        CCAFS SLC 40: Está en la costa de Florida.
        KSC LC 39A: También está en la costa de Florida, muy cerca de CCAFS.
        VAFB SLC 4E: Está en la costa de California.
        STLS: No está en la costa; está en el interior de Texas.
    La mayoría de los sitios de lanzamiento están muy cerca de la costa, excepto STLS que está en el interior. Esto sugiere una preferencia por sitios costeros debido a las consideraciones de seguridad mencionadas.

# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [12]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


In [14]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

# Crear una nueva columna 'marker_color' basada en el valor de 'class'
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Verificar los datos
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


*TODO:* Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


Let's first create a `MarkerCluster` object


In [15]:
# Crear un objeto MarkerCluster
marker_cluster = MarkerCluster().add_to(site_map)

# Añadir marcadores para cada registro de lanzamiento
for _, row in spacex_df.iterrows():
    # Extraer coordenadas y color del marcador
    coordinate = [row['Lat'], row['Long']]
    marker_color = row['marker_color']
    
    # Crear y añadir el marcador al cluster
    marker = folium.Marker(
        location=coordinate,
        icon=folium.Icon(color=marker_color),
        popup=f"{row['Launch Site']}: {'Success' if row['class'] == 1 else 'Failed'}"
    )
    marker.add_to(marker_cluster)

# Mostrar el mapa
site_map

In [16]:
#marker_cluster = MarkerCluster()


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [17]:
'''
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map
'''

"\n# Add marker_cluster to current site_map\nsite_map.add_child(marker_cluster)\n\n# for each row in spacex_df data frame\n# create a Marker object with its coordinate\n# and customize the Marker's icon property to indicate if this launch was successed or failed, \n# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']\nfor index, record in spacex_df.iterrows():\n    # TODO: Create and add a Marker cluster to the site map\n    # marker = folium.Marker(...)\n    marker_cluster.add_child(marker)\n\nsite_map\n"

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


### Observaciones y Patrones:

    Cape Canaveral Air Force Station (CCAFS) Launch Complex 40 (SLC-40):
        Este sitio de lanzamiento muestra una mayor concentración de marcadores verdes, indicando una alta tasa de éxito en sus lanzamientos.
        CCAFS SLC-40 es uno de los principales sitios utilizados por SpaceX y ha tenido numerosas misiones exitosas, lo que se refleja en la predominancia de marcadores verdes.

    Kennedy Space Center Launch Complex 39A (KSC LC-39A):
        Similar al CCAFS SLC-40, este sitio también muestra una alta proporción de marcadores verdes.
        KSC LC-39A es otro sitio crítico para SpaceX, utilizado para misiones importantes, incluidas las misiones tripuladas, y ha tenido una alta tasa de éxito.

    Vandenberg Air Force Base (VAFB) Space Launch Complex 4E (SLC-4E):
        VAFB SLC-4E muestra una mezcla de marcadores verdes y rojos, indicando una tasa de éxito moderada a alta.
        Este sitio se utiliza principalmente para lanzamientos hacia órbitas polares y heliosíncronas, y aunque ha tenido algunos fracasos, también ha visto muchos lanzamientos exitosos.

    Space Launch Complex 40 (SLC-40):
        Aunque no está en la costa, este sitio muestra una mezcla de éxito y fracasos, pero parece tener una menor cantidad de lanzamientos en comparación con los otros sitios.

## Conclusiones:
Preferencia por sitios costeros:

    La mayoría de los sitios con alta tasa de éxito (CCAFS SLC-40 y KSC LC-39A) están ubicados en la costa, lo que es consistente con la práctica de minimizar riesgos al lanzar sobre el océano.

Alta tasa de éxito en sitios principales:

    Los sitios de lanzamiento más utilizados (CCAFS SLC-40 y KSC LC-39A) muestran altas tasas de éxito. Esto podría deberse a una mejor infraestructura, más experiencia operativa y tal vez a condiciones más favorables.

Variedad en la tasa de éxito:

    La variabilidad en la tasa de éxito en VAFB SLC-4E sugiere que ciertos tipos de misiones (como aquellas hacia órbitas polares) pueden tener desafíos específicos que afectan las tasas de éxito.



# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [18]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [19]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [21]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# Coordenadas del sitio de lanzamiento CCAFS SLC-40
launch_site_lat = 28.562302
launch_site_lon = -80.577356

# Coordenadas del punto más cercano en la costa
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calcular la distancia
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
distance_coastline

0.5797120571581803

In [ ]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

In [22]:
# Coordenadas del punto más cercano en la costa
coastline_coordinate = [coastline_lat, coastline_lon]

# Crear y añadir el marcador en el punto más cercano en la costa
distance_marker = folium.Marker(
    coastline_coordinate,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
    )
)
site_map.add_child(distance_marker)

# Mostrar el mapa
site_map

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [23]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
#site_map.add_child(lines)

In [24]:
# Crear una PolyLine entre el sitio de lanzamiento y el punto de la costa
coordinates = [[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]]
lines = folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

# Mostrar el mapa
site_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [25]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site

# Coordenadas del sitio de lanzamiento CCAFS SLC-40
launch_site_lat = 28.562302
launch_site_lon = -80.577356

# Coordenadas de la ciudad más cercana (Melbourne)
city_lat = 28.0836
city_lon = -80.6081

# Coordenadas del ferrocarril más cercano (NASA Railroad)
railway_lat = 28.5721
railway_lon = -80.5853

# Coordenadas de la autopista más cercana (Samuel C Phillips Pkwy)
highway_lat = 28.5625
highway_lon = -80.5706

# Calcular las distancias
distance_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)
distance_railway = calculate_distance(launch_site_lat, launch_site_lon, railway_lat, railway_lon)
distance_highway = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)

In [26]:

# Añadir marcadores en los puntos de interés
city_coordinate = [city_lat, city_lon]
railway_coordinate = [railway_lat, railway_lon]
highway_coordinate = [highway_lat, highway_lon]

city_marker = folium.Marker(
    city_coordinate,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city),
    )
)
site_map.add_child(city_marker)

railway_marker = folium.Marker(
    railway_coordinate,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railway),
    )
)
site_map.add_child(railway_marker)

highway_marker = folium.Marker(
    highway_coordinate,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway),
    )
)
site_map.add_child(highway_marker)

# Crear líneas PolyLine entre el sitio de lanzamiento y los puntos de interés
city_line = folium.PolyLine(locations=[[launch_site_lat, launch_site_lon], city_coordinate], weight=1)
railway_line = folium.PolyLine(locations=[[launch_site_lat, launch_site_lon], railway_coordinate], weight=1)
highway_line = folium.PolyLine(locations=[[launch_site_lat, launch_site_lon], highway_coordinate], weight=1)

site_map.add_child(city_line)
site_map.add_child(railway_line)
site_map.add_child(highway_line)

# Mostrar el mapa
site_map

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


### ¿Están los sitios de lanzamiento en estrecha proximidad a los ferrocarriles?

Respuesta: Sí, los sitios de lanzamiento están en estrecha proximidad a los ferrocarriles.

Explicación: La cercanía a los ferrocarriles facilita el transporte de grandes componentes y materiales necesarios para los lanzamientos. Los ferrocarriles proporcionan una forma eficiente y segura de mover cargas pesadas a largas distancias.

### ¿Están los sitios de lanzamiento en estrecha proximidad a las autopistas?

Respuesta: Sí, los sitios de lanzamiento están en estrecha proximidad a las autopistas.

Explicación: Las autopistas cercanas aseguran un acceso rápido y eficiente para el personal y los suministros necesarios para las operaciones diarias y los lanzamientos. Facilitan la movilidad y la logística, permitiendo respuestas rápidas y transporte efectivo.

### ¿Están los sitios de lanzamiento en estrecha proximidad a la costa?

Respuesta: Sí, los sitios de lanzamiento están en estrecha proximidad a la costa.

Explicación: La proximidad a la costa es esencial por razones de seguridad. Al lanzar sobre el océano, se minimizan los riesgos para las áreas pobladas en caso de fallos durante el lanzamiento. Además, permite trayectorias de vuelo óptimas para ciertas órbitas.

### ¿Mantienen los sitios de lanzamiento cierta distancia de las ciudades?

Respuesta: Sí, los sitios de lanzamiento mantienen una cierta distancia de las ciudades.

Explicación: Mantener una distancia de las áreas urbanas es crucial para la seguridad. Los lanzamientos de cohetes implican riesgos significativos y la distancia ayuda a mitigar el impacto de posibles accidentes. También reduce el impacto del ruido y la contaminación en las zonas residenciales.

# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2022-01-01)


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
